# Covid19 - Counties Dataset
### The novel coronavirus has had huge implications to the world around us and how we live our day-to-day lives. In this notebook I'd like to explore how the coronavirus has spread over time especially around where I live in Northern New Jersey. As this area borders NYC, the new epicenter of the virus New Jersey is not surprisingly the state with the second most confirmed cases at the time of this analysis.

#### It's important to be mindful that these are confirmed cases which may or may not be indicative of the actual infections. Availability of testing kits and hospitals that allow testing for covid19 are scarce and it's hard to say if some parts of America test less than other places. 

#### Throughout the notebook I'll be commenting my train of thought which should translate to why I go a certain path with the analysis. The data used here is provided through the New York Times. 

In [18]:
# import libraries
import pandas as pd
import numpy as np

# import data
data = pd.read_csv('dataset/us-counties.csv')

# preview the data
print('Number of data points:', data.shape[0])
data.head()

Number of data points: 30843


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [23]:
# check if empty entries exist
data.isnull().sum()

date      0
county    0
state     0
cases     0
deaths    0
dtype: int64

There seems to be some blank entries for the column fips which is a unique code for each county. As we'll only be doing analysis on the county and states we can omit the fips column entirely for the purposes of this analysis.

In [21]:
# remove fips column
data.drop('fips', axis=1, inplace=True)

Let's run some explaratory analysis now we know this data is in a usable state.

In [31]:
# min/max dates in dataset
print('Min date:', min(data.date))
print('Max date:', max(data.date))
print('')

# number of states in dataset
print('Number of distinct states in dataset:', data.state.nunique())

Min date: 2020-01-21
Max date: 2020-04-03

Number of distinct states in dataset: 55


Um.. 55 distinct states?? We'll have to list them out and vett the list to see what's included. In the case there's typos in the state names we'll have to do additional data cleaning.

In [45]:
for state in sorted(data.state.unique()):
    print(state)

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Guam
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Northern Mariana Islands
Ohio
Oklahoma
Oregon
Pennsylvania
Puerto Rico
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virgin Islands
Virginia
Washington
West Virginia
Wisconsin
Wyoming


District of Columbia, Guam, Northern Mariana Islands and Virgin Islands are not typical states, but we'll keep them in the dataset. Let's rank the number of cases and deaths for each of these states.

As we will running analysis on multiple levels of data we'll make a function for adding some useful statistics for the table.

In [71]:
def create_rate_cols(tdf):
    '''Adds death rate, cases percent of total and deaths percent of total columns to a dataframe.'''
    df = tdf.copy()
    sum_cases = sum(df.cases)
    sum_deaths = sum(df.deaths)
    
    # add columns
    df['death_rate'] = df.deaths / df.cases
    df['cases perc. of total'] = df.cases / sum_cases
    df['deaths perc. of total'] = df.deaths / sum_deaths
    
    cols = ['death_rate', 'cases perc. of total', 'deaths perc. of total']
    
    # convert decimals to percentage
    for col in cols:
        df[col] = df[col].astype(float).map('{:.2%}'.format)
    
    return df

In [73]:
# sum the cases and deaths per state
state_cd = data.groupby('state').sum()[['cases', 'deaths']]

# add stats columns
state_cd = create_rate_cols(state_cd)

# print dataframe sorted by death cases
state_cd.sort_values('cases', ascending=False)

,cases,deaths,death_rate,cases perc. of total,deaths perc. of total
state,,,,,
New York,743198,12602,1.70%,40.61%,35.48%
New Jersey,170274,2762,1.62%,9.30%,7.78%
California,89145,1847,2.07%,4.87%,5.20%
Michigan,71954,2198,3.05%,3.93%,6.19%
Washington,63368,3193,5.04%,3.46%,8.99%
Florida,62627,902,1.44%,3.42%,2.54%
Massachusetts,61632,809,1.31%,3.37%,2.28%
Illinois,57310,971,1.69%,3.13%,2.73%
Louisiana,54795,2098,3.83%,2.99%,5.91%


New York has almost 41% of all coronavirus confirmed cases and New York and New Jersey combined have nearly half of all of America's infections. I live right on the border of NY and NJ which is the most heavily populated area in NJ so I assume most of the coronavirus cases are going to be in my county (which is Bergen). For NY I assume most of their cases is in NYC.

In [78]:
# create dataset of only New York and New Jersey
nyj = data[data.state.isin(['New York', 'New Jersey'])]
nj = data[data.state=='New Jersey']
ny = data[data.state=='New York']

# let's first look at the counties in New Jersey
nj_county = nj.groupby('county').sum().sort_values('cases', ascending=False)

# create rate columns
nj_county = create_rate_cols(nj_county)

# print dataframe
nj_county

,cases,deaths,death_rate,cases perc. of total,deaths perc. of total
county,,,,,
Unknown,33043,85,0.26%,19.41%,3.08%
Bergen,28562,627,2.20%,16.77%,22.70%
Essex,16691,479,2.87%,9.80%,17.34%
Hudson,13842,186,1.34%,8.13%,6.73%
Union,12418,179,1.44%,7.29%,6.48%
Middlesex,11981,222,1.85%,7.04%,8.04%
Passaic,11078,125,1.13%,6.51%,4.53%
Monmouth,10750,206,1.92%,6.31%,7.46%
Ocean,9236,173,1.87%,5.42%,6.26%


Looks like there's an unknown category for county which may be due to lack of information from the source. We'll remove the unknown county entries for now to not skew the percentages, but also be aware we're removing some of the data from the dataset.

In [81]:
# recreate dataframe with New Jersey counties
nj_county = nj.groupby('county').sum().sort_values('cases', ascending=False)

# remove unknown county label
nj_county = nj_county[nj_county.index!='Unknown']

# create rate columns
nj_county = create_rate_cols(nj_county)

# print dataframe
nj_county

,cases,deaths,death_rate,cases perc. of total,deaths perc. of total
county,,,,,
Bergen,28562,627,2.20%,20.81%,23.42%
Essex,16691,479,2.87%,12.16%,17.89%
Hudson,13842,186,1.34%,10.09%,6.95%
Union,12418,179,1.44%,9.05%,6.69%
Middlesex,11981,222,1.85%,8.73%,8.29%
Passaic,11078,125,1.13%,8.07%,4.67%
Monmouth,10750,206,1.92%,7.83%,7.70%
Ocean,9236,173,1.87%,6.73%,6.46%
Morris,7506,194,2.58%,5.47%,7.25%


So the first known county of most cases is Bergen county.. where I live. It is a little more spread out than I would've guessed however as I thought the percentage of cases in Bergen county would've been much more than 20.81% of total cases in New Jersey.

Let's now look at New York at their counties.

In [82]:
# recreate dataframe with New Jersey counties
ny_county = ny.groupby('county').sum().sort_values('cases', ascending=False)

# remove unknown county label
ny_county = ny_county[ny_county.index!='Unknown']

# create rate columns
ny_county = create_rate_cols(ny_county)

# print dataframe
ny_county

,cases,deaths,death_rate,cases perc. of total,deaths perc. of total
county,,,,,
New York City,423482,9946,2.35%,56.98%,82.92%
Westchester,102386,253,0.25%,13.78%,2.11%
Nassau,82243,600,0.73%,11.07%,5.00%
Suffolk,62543,552,0.88%,8.42%,4.60%
Rockland,26828,195,0.73%,3.61%,1.63%
Orange,15349,102,0.66%,2.07%,0.85%
Erie,4547,89,1.96%,0.61%,0.74%
Dutchess,4532,38,0.84%,0.61%,0.32%
Monroe,2986,66,2.21%,0.40%,0.55%


New York City has 57% of the cases in the state of New York and 423,482 confirmed cases which is a staggering number. A quick look at the census data provided by NYC open data says for 2020 there's approximately 8,550,971 people living in NYC. 

This means 4.95% of the people living in NYC today are confirmed to have covid19.